### Pandas Lab -- Cleaning, Merging, & Grouping

This lab is designed to introduce students to common use cases for Pandas when working with data:

 - Creating new information out of your existing data set
 - Merging, concatenating, and joining different data sources
 - Grouping -- With both time & non-time based data

### Section I: Creating Data Out of Your Existing Columns

Go ahead and create the following columns in your dataset.

In [151]:
import pandas as pd
import numpy as np
# read in the file
df = pd.read_excel('../data/superstore.xls', index_col='Row ID')

**Column 1:**

  - **Column Name:** Profitable
  - **Values:** `True` if `Profit` > 0, `False` if not.

In [71]:
# your answer here
df['Profitable'] = np.where(df['Profit'] > 0, True, False)

**Column 2:**

 - **Column Name:** Expected Ship Time
 - **Values:**
   - `0` if `Ship Mode` == `Same Day`
   - `2` if `Ship Mode` == `First Class`
   - `3` if `Ship Mode` == `Second Class`
   - `6` if `Ship Mode` == `Standard Class`
   - `-1` if none of the above.

In [72]:
# your answer here
conditions = [
    df['Ship Mode'] == 'Same Day',
    df['Ship Mode'] == 'First Class',
    df['Ship Mode'] == 'Second Class',
    df['Ship Mode'] == 'Standard Class'
]

results = [0, 2, 3, 6]

df['Expected Ship Time'] = np.select(conditions, results, -1)

**Column 3:**

 - **Column Name:** Actual Ship Time
 - **Values:**
   - `Ship Date` - `Order Date`
 - **Note:** When you subtract these columns, your column will be a **time delta**.  See if you can use the `dt` attribute to convert these values into an integer.  Ie, if your value reads `3 days`, you want that to be 3 instead.  You can read more about different time periods in pandas here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [73]:
# your answer here
df['Actual Ship Time'] = (df['Ship Date'] - df['Order Date']).dt.days

**Column 4:**

 - **Column Name:** Late
 - **Values:** `True` if `Actual Ship Time` > `Expected Ship Time`, `False` otherwise

In [74]:
# your answer here
df['Late'] = df['Actual Ship Time'] > df['Expected Ship Time']

### Section II: Merging Dataframes

This excel spreadsheet has 3 separate sheets.  Look up the documentation on the `pd.read_excel` method on how to load in the other two.  

After that, merge the other two dataframes into your original one, and make sure your original dataset now has the following columns:

 - **Salesperson:** This is the Salesperson in charge of each region.
 - **Returned:** This details whether or not the order was returned.  Fill in null values with the value `no`.
 
Use the `rename()` method to create the Salesperson column label.

**Important:** We want to keep all of the rows in the dataset we first loaded in.  After each merge, it's a good idea to make sure your dataset hasn't shrunk, which will happen if you don't choose the right merge type.  Make sure you have 9,994 rows when you're finished!

In [75]:
# first, we'll import the other two sheets as different dataframes
df2 = pd.read_excel('../data/superstore.xls', sheet_name=1)
df3 = pd.read_excel('../data/superstore.xls', sheet_name=2)

In [76]:
#we're going to do a LEFT join, to keep all the values in the original dataframe
df = df.merge(df2, on='Order ID', how='left')

In [77]:
# notice we have lots of empty values when we do the merge
df['Returned'].value_counts(dropna=False)

NaN    9194
Yes     800
Name: Returned, dtype: int64

In [78]:
# so we'll fill these in
df['Returned'].fillna('No', inplace=True)

In [79]:
# and we'll merge the other one in
# you could do a left or inner merge and still get the same results
df = df.merge(df3, on='Region', how='inner')

In [80]:
# takes a dictionary as an argument
# 'old label': 'new label' -- can be used to rename rows or columns
df.rename({'Person': 'Salesperson'}, axis=1, inplace=True)

### Section III: Grouping

Use the `groupby` or `resample` method to answer the following questions.

**Question 1:** What salesperson had the highest average sales amount? 

In [111]:
# It was Cassandra
df.groupby('Salesperson')['Sales'].mean()

Salesperson
Anna Andreadi        226.493233
Cassandra Brandow    241.803645
Chuck Magee          238.336110
Kelly Williams       215.772661
Name: Sales, dtype: float64

**Question 2:** Within each ship mode, compare how likely late orders were to be profitable or not

In [86]:
# The answer, for the most part -- looks like no difference, except for Same Day
df.groupby(['Ship Mode', 'Late'])['Profitable'].mean()

Ship Mode       Late 
First Class     False    0.795405
                True     0.814103
Same Day        False    0.813102
                True     0.916667
Second Class    False    0.845415
                True     0.827500
Standard Class  False    0.794651
                True     0.805153
Name: Profitable, dtype: float64

**Question 3:** What ship mode had the most consistently on time orders?

In [83]:
# Same Day
df.groupby('Ship Mode')['Late'].mean()

Ship Mode
First Class       0.405722
Same Day          0.044199
Second Class      0.411311
Standard Class    0.104055
Name: Late, dtype: float64

**Question 4:** For each sales person, get their average, median, max, and count of their sales.

In [87]:
# your answer here
df.groupby('Salesperson')['Sales'].agg(['mean', 'median', 'max', 'count'])

,mean,median,max,count
Salesperson,,,,
Anna Andreadi,226.493233,60.840,13999.960,3203
Cassandra Brandow,241.803645,54.594,22638.480,1620
Chuck Magee,238.336110,54.900,11199.968,2848
Kelly Williams,215.772661,45.980,17499.950,2323


**Question 5:** Group your dataset according to  `Region`, and `Category`, and then call the `describe()` method to get the summary statistics for each subgroup.

In [113]:
# your answer here
df.groupby(['Region', 'Category']).describe()

Actual Ship Time                                     \
                                   count      mean       std  min  25%  50%   
Region  Category                                                              
Central Furniture                  481.0  3.979210  1.713787  0.0  3.0  4.0   
        Office Supplies           1422.0  4.120253  1.707383  0.0  3.0  4.0   
        Technology                 420.0  3.938095  1.586331  0.0  3.0  4.0   
East    Furniture                  601.0  3.850250  1.747628  0.0  3.0  4.0   
        Office Supplies           1712.0  3.937500  1.724834  0.0  3.0  4.0   
        Technology                 535.0  3.882243  1.720977  0.0  3.0  4.0   
South   Furniture                  332.0  4.024096  1.690392  0.0  3.0  4.0   
        Office Supplies            995.0  3.955779  1.723916  0.0  3.0  4.0   
        Technology                 293.0  3.890785  1.836190  0.0  3.0  4.0   
West    Furniture                  707.0  3.881188  1.818235  0.0  2.0  4.0   
        Office Supplies           1897.0  3.936742  1.804590  0.0  3.0  4.0   
        Technology                 599.0  3.964942  1.809610  0.0  3.0  4.0   

                                  Discount              ...     Quantity  \
                         75%  max    count      mean    ...          75%   
Region  Category                                        ...                
Central Furniture        5.0  7.0    481.0  0.297380    ...          5.0   
        Office Supplies  5.0  7.0   1422.0  0.252743    ...          5.0   
        Technology       5.0  7.0    420.0  0.133095    ...          5.0   
East    Furniture        5.0  7.0    601.0  0.154077    ...          5.0   
        Office Supplies  5.0  7.0   1712.0  0.142932    ...          5.0   
        Technology       5.0  7.0    535.0  0.143364    ...          5.0   
South   Furniture        5.0  7.0    332.0  0.121536    ...          5.0   
        Office Supplies  5.0  7.0    995.0  0.167437    ...          5.0   
        Technology       5.0  7.0    293.0  0.107850    ...          5.0   
West    Furniture        5.0  7.0    707.0  0.131400    ...          5.0   
        Office Supplies  5.0  7.0   1897.0  0.093358    ...          5.0   
        Technology       5.0  7.0    599.0  0.133890    ...          5.0   

                                Sales                                  \
                          max   count        mean          std    min   
Region  Category                                                        
Central Furniture        14.0   481.0  340.534644   463.964050  1.892   
        Office Supplies  14.0  1422.0  117.458801   474.295578  0.444   
        Technology       12.0   420.0  405.753124  1059.529951  1.980   
East    Furniture        14.0   601.0  346.574383   547.628260  2.960   
        Office Supplies  14.0  1712.0  120.044425   330.587488  0.852   
        Technology       14.0   535.0  495.278469  1114.810815  2.970   
South   Furniture        14.0   332.0  353.309289   513.186098  2.784   
        Office Supplies  14.0   995.0  126.282727   357.811953  1.167   
        Technology       14.0   293.0  507.753952  1569.548933  1.584   
West    Furniture        14.0   707.0  357.302325   485.240433  3.480   
        Office Supplies  14.0  1897.0  116.422377   360.236892  1.080   
        Technology       14.0   599.0  420.687533   830.228235  0.990   

                                                                 
                              25%      50%       75%        max  
Region  Category                                                 
Central Furniture        34.50400  191.058  459.9200   3504.900  
        Office Supplies  10.36800   24.046   69.5640   9892.740  
        Technology       62.37750  158.376  401.1975  17499.950  
East    Furniture        51.96800  172.110  396.8020   4416.174  
        Office Supplies  11.67225   27.108   78.8640   4663.736  
        Technology       63.90000  158.376  479.9510  11199.968  
South  

**Question 6:** Use the `Resample()` method to get the sum of quarterly sales.

In [37]:
# your answer here
df.resample('Q', on='Order Date')['Sales'].sum()

Order Date
2014-03-31     74447.7960
2014-06-30     86538.7596
2014-09-30    143633.2123
2014-12-31    179627.7302
2015-03-31     68851.7386
2015-06-30     89124.1870
2015-09-30    130259.5752
2015-12-31    182297.0082
2016-03-31     93237.1810
2016-06-30    136082.3010
2016-09-30    143787.3622
2016-12-31    236098.7538
2017-03-31    123144.8602
2017-06-30    133764.3720
2017-09-30    196251.9560
2017-12-31    280054.0670
Freq: Q-DEC, Name: Sales, dtype: float64

**Question 7:** What quarter had the highest total sales amount?

In [44]:
# this would give you the maximum value
df.resample('Q', on='Order Date')['Sales'].sum().max()

280054.067

In [45]:
# if you wanted to get the actual Quarter that his happened in , do this
df.resample('Q', on='Order Date')['Sales'].sum().idxmax()

Timestamp('2017-12-31 00:00:00', freq='Q-DEC')

In [61]:
# and if you wanted both......
grouping = df.resample('Q', on='Order Date')['Sales'].sum().reset_index()
max_idx  = grouping['Sales'].idxmax()
grouping.iloc[max_idx]

Order Date    2017-12-31 00:00:00
Sales                      280054
Name: 15, dtype: object

**Question 8:** See if you can use the `groupby` method to get a list of yearly sales for each region inside the dataset.

**Hint:** Try using the `dt` attribute of the `Order Date` column

In [105]:
df.groupby(['Region', df['Order Date'].dt.year])['Sales'].sum()

Region   Order Date
Central  2014          103838.1646
         2015          102874.2220
         2016          147429.3760
         2017          147098.1282
East     2014          128680.4570
         2015          156332.0570
         2016          180685.8220
         2017          213082.9040
South    2014          103845.8435
         2015           71359.9805
         2016           93610.2235
         2017          122905.8575
West     2014          147883.0330
         2015          139966.2495
         2016          187480.1765
         2017          250128.3655
Name: Sales, dtype: float64

**Bonus:** Creating summary statistics with a `groupby` statement.  

Lots of times it's very useful to be able to create a summary statistic for a particular category to compare with individual samples.

For example, if you were doing fraud detection, and someone were making a purchase at a 7-11, a transaction amount of $175 would be unusually large for someone making a purchase at a convenience store, essentially setting off a red flag that the transaction might be suspicious.

Creating such comparisons is easily done using the `groupby` method and then merging it back into the original dataframe.  

For example, if you wanted to compare every single purchase amount with the average amount for that category, you could do it in the following way:

In [152]:
# create the grouping
cat_grouping = df.groupby('Category')[['Sales']].mean()
# this step is mostly just to make the merged dataframe more tidy
cat_grouping.rename({'Sales': 'Cat_Average'}, axis=1, inplace=True)

In [153]:
# join them
df = df.merge(cat_grouping, left_on='Category', right_index=True)

In [154]:
# and now we can see each purchase amount compared to the average amt
# for that category
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Cat_Average
Row ID,,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,349.834887
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,349.834887
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,349.834887
6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,349.834887
11,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092,349.834887


So for instance, if wanted to ask ourselves, "which customers consistently punch above their weight when it comes to the actual items that they buy?"

We could easily do the following:

In [155]:
# turn the difference between the two values into a percent
df['Cat Difference'] = ((df['Sales'] / df['Cat_Average']) - 1) * 100

In [157]:
# now group and sort the values
df.groupby('Customer Name')['Cat Difference'].mean().sort_values(ascending=False)

Customer Name
Mitch Willingham        845.569285
Christopher Martinez    636.313301
Andy Reiter             449.654450
Adrian Barton           418.548520
Sanjit Chand            386.906876
Amy Cox                 363.031889
Yoseph Carroll          344.794116
Yana Sorensen           317.572571
Sean Miller             308.018602
Tamara Chand            304.928500
Alex Avila              296.561353
Greg Maxwell            292.917810
Grant Thornton          291.980410
Jane Waco               272.432920
Tom Ashbrook            270.172385
Paul Knutson            244.824972
Robert Dilbeck          239.904239
Gary Hwang              231.081468
Ken Lonsdale            216.273514
Dennis Pardue           213.919098
Justin Hirsh            205.806859
Stefanie Holloman       199.625920
Cathy Prescott          194.089483
Bill Shonely            194.068945
Erica Smith             193.504802
Neil Ducich             174.357173
Adam Bellavance         174.177216
Hunter Lopez            170.364451
Karen 

Clearly, not all customers are created equal.

**Your Turn:** Using a similar methodology as above, figure out the 10 customers who are the most profitable on average, when compared to the subcategory that they purchased from.

If you wanted, you could also limit this to customers who only made a minimum number of purchases as well.

In [159]:
# create the grouping
subcat_grouping = df.groupby('Sub-Category')[['Profit']].mean()
# do some renaming to make things tidier
subcat_grouping.rename({'Profit': 'Subcat Avg'}, axis=1, inplace=True)
# merge
df = df.merge(subcat_grouping, left_on='Sub-Category', right_index=True)

In [160]:
# compare differences between the two columns, as a percentage
df['Subcat Diff'] = ((df['Profit']/df['Subcat Avg']) - 1) *100

In [162]:
# now group customers according to the column you just made
top_custs = df.groupby('Customer Name')['Subcat Diff'].mean()
# and sort the values + show the top 10 -- Mitch is a high roller.....
top_custs.sort_values(ascending=False).head(10)

Customer Name
Mitch Willingham        2404.056459
Christopher Martinez    1862.436871
Jim Radford             1628.279903
Andy Reiter             1373.983641
Adrian Barton           1337.826612
Sanjit Chand            1236.837470
Alex Avila              1202.507813
Bill Shonely             880.808891
Sanjit Engle             750.951436
Jenna Caffey             728.799638
Name: Subcat Diff, dtype: float64